In [ ]:
pwd

'/content'

In [11]:
import pandas as pd
import numpy as np
data=pd.read_csv("//content//ner_data.csv")
data.head(5)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [12]:
import pandas as pd
import numpy as np

In [13]:
pwd

'/content'

In [14]:
data=pd.read_csv("//content//ner_data.csv")
data.head(6)
## Words are splitted and All the columns represent X label except Tag and Tag is Y label

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O


In [15]:
##Data Preparation for Neural Networks

In [16]:
from itertools import chain

In [17]:
def get_dict_map(data, token_or_tag):
    tok2idx = {}
    idx2tok = {}
    
    if token_or_tag == 'token':
        vocab = list(set(data['Word'].to_list()))
    else:
        vocab = list(set(data['Tag'].to_list()))
    
    idx2tok = {idx:tok for  idx, tok in enumerate(vocab)}
    tok2idx = {tok:idx for  idx, tok in enumerate(vocab)}
    return tok2idx, idx2tok
token2idx, idx2token = get_dict_map(data, 'token')
tag2idx, idx2tag = get_dict_map(data, 'tag')

In [18]:
data['Word_idx']=data['Word'].map(token2idx)
data['Tag_idx']=data['Tag'].map(tag2idx)
data_fillna=data.fillna(method='ffill',axis=0)

In [19]:
##Groupby and collect columns

In [20]:
data_group = data_fillna.groupby(
['Sentence #'],as_index=False
)['Word', 'POS', 'Tag', 'Word_idx', 'Tag_idx'].agg(lambda x: list(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
data.head(10)

,Sentence #,Word,POS,Tag,Word_idx,Tag_idx
0,Sentence: 1,Thousands,NNS,O,17504,11
1,NaN,of,IN,O,27644,11
2,NaN,demonstrators,NNS,O,9960,11
3,NaN,have,VBP,O,5501,11
4,NaN,marched,VBN,O,4847,11
5,NaN,through,IN,O,4899,11
6,NaN,London,NNP,B-geo,33721,12
7,NaN,to,TO,O,20055,11
8,NaN,protest,VB,O,11802,11
9,NaN,the,DT,O,34292,11


In [22]:
data.shape

(1048575, 6)

In [23]:
## Using LSTM -splitting Train and test dataset and it accepts sequence of same length

In [24]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [25]:
# getting maximum token and tag length

In [26]:
n_token=len(list(set(data['Word'].to_list())))
n_tag=len(list(set(data['Tag'].to_list())))

In [27]:
print(n_token)
print(n_tag)

35164
17


In [28]:
#Pad tokens (X Var)

In [29]:
tokens=data_group['Word_idx'].to_list()
maxlen=max([len(s) for s in tokens])
pad_tokens=pad_sequences(tokens,maxlen=maxlen,dtype='int32',padding='post',value=n_token-1)

In [30]:
# Pad tokens (y var) and convert it into one hot encoding

In [32]:
tags=data_group['Tag_idx'].to_list()
pad_tags=pad_sequences(tags,maxlen,dtype='int32',padding='post',value=tag2idx["O"])
n_tags=len(tag2idx)
pad_tags=[to_categorical(i,num_classes=n_tags)for i in pad_tags]

In [33]:
## Split train,test and validation set

In [34]:
tokens_,test_tokens,tags_,test_tags=train_test_split(pad_tokens,pad_tags,test_size=0.1,train_size=0.9,random_state=2020)
train_tokens,val_tokens,train_tags,val_tags=train_test_split(tokens_,tags_,test_size=0.25,train_size=0.75,random_state=2020)

In [36]:
print(
        'train_tokens length:', len(train_tokens),
        '\ntest_tokens length:', len(test_tokens),
        '\ntest_tags:', len(test_tags),
        '\nval_tokens:', len(val_tokens),
        '\nval_tags:', len(val_tags),
    )

train_tokens length: 32372 
test_tokens length: 4796 
test_tags: 4796 
val_tokens: 10791 
val_tags: 10791


In [37]:
import numpy as np
import tensorflow
from tensorflow.keras import Sequential,Model,Input
from tensorflow.keras.layers import LSTM,Embedding,Dense,TimeDistributed,Dropout,Bidirectional
from tensorflow.keras.utils import plot_model
from numpy.random import seed
seed(1)
tensorflow.random.set_seed(2)

In [38]:
input_dim = len(list(set(data['Word'].to_list())))+1
output_dim = 64
input_length = max([len(s) for s in data_group['Word_idx'].tolist()])
n_tags = len(tag2idx)

In [39]:
def get_bilstm_lstm_model():
  model = Sequential()
# Add Embedding layer
  model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length))
  # Add bidirectional LSTM
  model.add(Bidirectional(LSTM(units=output_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode = 'concat'))
  # Add LSTM
  model.add(LSTM(units=output_dim, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))
  # Add timeDistributed Layer
  model.add(TimeDistributed(Dense(n_tags, activation="relu")))
  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.summary()
  return model

In [6]:
def train_model(X, y, model):
    loss = list()
    for i in range(25):
        # fit model for one epoch on this sequence
        hist = model.fit(X, y, batch_size=1000, verbose=1, epochs=1, validation_split=0.2)
        loss.append(hist.history['loss'][0])
    return loss

In [40]:
results = pd.DataFrame()
model_bilstm_lstm = get_bilstm_lstm_model()
plot_model(model_bilstm_lstm)
results['with_add_lstm'] = train_model(train_tokens, np.array(train_tags), model_bilstm_lstm)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 104, 64)           2250560   
_________________________________________________________________
bidirectional (Bidirectional (None, 104, 128)          66048     
_________________________________________________________________
lstm_1 (LSTM)                (None, 104, 64)           49408     
_________________________________________________________________
time_distributed (TimeDistri (None, 104, 17)           1105      
Total params: 2,367,121
Trainable params: 2,367,121
Non-trainable params: 0
_________________________________________________________________
26/26 [==============================] - 139s 5s/step - loss: nan - accuracy: 0.0035 - val_loss: nan - val_accuracy: 0.0032


In [41]:
import spacy
from spacy import displacy
nlp=spacy.load("en_core_web_sm")
text=nlp('Hi My name is Bharathidasan \n im from India,Tamilnadu,Chennai \n i want to work with HCL')
displacy.render(text,style='ent',jupyter=True)